In [1]:
import torch
from torch import optim
import torch.nn as nn
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import torchvision
import pandas as pd
import copy
import random
import pickle
import torch.nn.functional as F
from torch.nn import Linear

In [2]:
#load the processed split data
def load_df(name):
    return pd.read_pickle('data/'+name+'.pkl')

[x_train, y_train, x_test, y_test, x_val, y_val] = map(load_df, ['x_train','y_train','x_test','y_test', 'x_val','y_val']) 

In [3]:
def typing(df):
    sequences=df.astype(np.float32).to_numpy().tolist()
    dataset = torch.stack([torch.tensor(s).unsqueeze(1).float() for s in sequences])
    return dataset

x_train, y_train, x_test, y_test, x_val, y_val = map(typing, [x_train, y_train.drop('sample_collection_site',axis=1), x_test, y_test.drop('sample_collection_site',axis=1), x_val, y_val.drop('sample_collection_site',axis=1)])

In [12]:
x_train[:,[1,4]].shape

torch.Size([1052, 2, 1])

In [4]:
from torch.utils.data import DataLoader,TensorDataset
from torch import Tensor

#y_train_ohe = y_train.drop('sample_collection_site', axis=1)
dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

#y_val_ohe = y_val.drop('sample_collection_site', axis=1)
dataset_val = TensorDataset(x_val, y_val)
val_loader = DataLoader(dataset_val, batch_size=8, shuffle=False, num_workers=2, pin_memory=True)


In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train).astype(np.float64)
dataset = TensorDataset(Tensor(x_train), Tensor(y_train_ohe.astype(np.float32).to_numpy()))
train_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=2, pin_memory=True)


In [17]:
next(iter(train_loader)).shape

torch.Size([4, 19221])

In [3]:
from torch.utils.data import DataLoader,TensorDataset
from torch import Tensor

y_train_ohe = y_train.drop('sample_collection_site', axis=1)
dataset = TensorDataset(Tensor(x_train.astype(np.float64).to_numpy()), Tensor(y_train_ohe.astype(np.float64).to_numpy()))
train_loader = DataLoader(dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

y_val_ohe = y_val.drop('sample_collection_site', axis=1)
dataset_val = TensorDataset(Tensor(x_val.astype(np.float64).to_numpy()), Tensor(y_val_ohe.astype(np.float64).to_numpy()))
val_loader = DataLoader(dataset_val, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)


In [4]:
class EnCl(nn.Module):
    def __init__(self, input_size=19221, drop_rate=0.2, num_class=39):
        super(EnCl, self).__init__()
        self.encode = nn.Sequential(
            nn.Linear(input_size, input_size//2),
            nn.ReLU(inplace=True),
            nn.Dropout(drop_rate),
            nn.Linear(input_size//2, input_size//2//4),
            nn.ReLU(inplace=True),
            nn.Dropout(drop_rate),
            nn.Linear(input_size//2//4, input_size//2//4//4),
            nn.ReLU(inplace=True),
            #nn.relu(drop_rate),
            nn.Linear(input_size//2//4//4, input_size//2//4//4//8),
            nn.ReLU(inplace=True),
            #nn.relu(drop_rate),
            nn.Linear(input_size//2//4//4//8, num_class)
        )
        
        self.classify = nn.Sequential(
            nn.BatchNorm1d(num_class),
            nn.ReLU(inplace=True),
            nn.Linear(num_class, num_class),
            nn.ReLU(inplace=True),
            nn.Linear(num_class, num_class),
            nn.ReLU(inplace=True),
            nn.Linear(num_class, num_class),
            #nn.ReLU(True),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        code = self.encode(x)
        x = self.classify(code)
        return x

In [4]:
class Encoder(nn.Module):
    def __init__(self, input_size=18252, num_class=39, drop_rate=0.2):
        super(Encoder, self).__init__()
        self.drop_rate = drop_rate
        self.enc1 = nn.Linear(input_size, input_size//2)
        self.enc2 = nn.Linear(input_size//2, input_size//2//4)
        self.enc3 = nn.Linear(input_size//2//4, input_size//2//4//4)
        self.enc4 = nn.Linear(input_size//2//4//4, input_size//2//4//4//8)
        self.enc5 = nn.Linear(input_size//2//4//4//8, num_class)
    def forward(self, features):
        x = torch.relu(torch.dropout(self.enc1(features), p=self.drop_rate, train=True))
        x = torch.relu(torch.dropout(self.enc2(x), p=self.drop_rate, train=True))
        x = torch.relu(torch.dropout(self.enc3(x), p=self.drop_rate, train=True))
        x = torch.relu(torch.dropout(self.enc4(x), p=self.drop_rate, train=True))
        #x = torch.dropout(torch.relu(self.enc5(x)), p=drop_rate)
        code = self.enc5(x)
        return code

class MC_LR(nn.Module):
    def __init__(self, input_size=18252, num_class=39, drop_rate=0.2):
        super(MC_LR, self).__init__()
        self.fc1 = nn.Linear(num_class, num_class)
        self.fc2 = nn.Linear(num_class, num_class)
        self.fc3 = nn.Linear(num_class, num_class)
    def forward(self, code):
        x = torch.relu(self.fc1(code))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        cl = torch.softmax(x,dim=1)
        return cl

class Enc_MC_LR(nn.Module):
    def __init__(self):
        super(Enc_MC_LR, self).__init__()
        self.encoder = Encoder()
        self.regressor = MC_LR()
    def forward(self, features):
        latent = self.encoder(features)
        return self.regressor(latent)

In [11]:
19221//16

1201

In [4]:
class EnClS(nn.Module):
    def __init__(self, input_size=18252, drop_rate=0, num_class=39):
        super(EnClS, self).__init__()
        self.encode = nn.Sequential(
            nn.Linear(input_size, input_size//2),
            nn.ReLU(inplace=True),
            nn.Dropout(drop_rate),
            nn.Linear(input_size//2, input_size//16),
            nn.ReLU(inplace=True),
            nn.Dropout(drop_rate),
            nn.Linear(input_size//16, num_class)
        )
        
        self.classify = nn.Sequential(
            #nn.BatchNorm1d(num_class),
            nn.ReLU(inplace=True),
            nn.Linear(num_class, num_class),
            #nn.ReLU(True),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        code = self.encode(x)
        x = self.classify(code)
        return x

In [18]:
from torchsummary import summary
print(summary(EnClS().to('cuda'),(1054,18252)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1           [-1, 1054, 9126]     166,576,878
              ReLU-2           [-1, 1054, 9126]               0
           Dropout-3           [-1, 1054, 9126]               0
            Linear-4           [-1, 1054, 1140]      10,404,780
              ReLU-5           [-1, 1054, 1140]               0
           Dropout-6           [-1, 1054, 1140]               0
            Linear-7             [-1, 1054, 39]          44,499
              ReLU-8             [-1, 1054, 39]               0
            Linear-9             [-1, 1054, 39]           1,560
          Softmax-10             [-1, 1054, 39]               0
Total params: 177,027,717
Trainable params: 177,027,717
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 73.39
Forward/backward pass size (MB): 248.91
Params size (MB): 675.3

In [5]:
def train_valid(model, device, train_loader, valid_loader, l1_lambda=0.01, class_num=39, input_len=18252, n_epochs=100):
  #optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.05)
  optimizer = optim.Adam(model.parameters(), lr=1e-5)
  criterion = nn.CrossEntropyLoss()
  losses = []
  min_val_loss = np.inf
  for epoch in tqdm(range(n_epochs)):

      model.train()
      loss = 0
      this = 0
      for batched, label in train_loader:
          this += 1
          #reshape
          #batch_features = batch_features.view(-1, input_len).to(device)
          # reset the gradients back to zero
          # PyTorch accumulates gradients on subsequent backward passes
          optimizer.zero_grad()
          # compute reconstructions
          #codes = model.encode(batch_features)#[1]
          #outputs = model.classify(codes)
          #sm = nn.Softmax(dim=1)
          #outputs = sm(codes)
          outputs = model(batched.to(device))
          # print(batch_features)
          # print(batch_features.shape)
          # print(codes)
          # print(codes.shape)
          #print(outputs)
          # print(outputs.shape)
          # print(label)
          # print(label.shape)
          #print(s)
          #compute training reconstruction loss
          # print(this)
          # print(outputs)
          # print(outputs.shape)
          #label = label.view(-1, class_num).to(device)
          train_loss = criterion(outputs, label.to(device))
          # compute accumulated gradients
          train_loss.backward()
          # perform parameter update based on current gradients
          optimizer.step()
          # add the mini-batch training loss to epoch loss
          loss += (train_loss.item()) #+ l1_lambda*sum(p.abs().sum() for p in model.parameters()))

      model.eval()
      val_loss = 0
      #with torch.no_grad():
      for feats, lv in valid_loader:
          feats = feats.to(device)
          target = model(feats)
          l = criterion(target, lv.to(device))
          val_loss += (l.item())# + l1_lambda*sum(p.abs().sum() for p in model.parameters()))

      # compute the epoch training loss
      loss = loss / len(train_loader)
      val_loss = val_loss / len(val_loader)
      losses.append((loss, val_loss))
      # display the epoch training loss
      print("epoch : {}/{}, loss = {:.6f}, val_loss = {:.6f}".format(epoch + 1, n_epochs, loss, val_loss))
      
      if min_val_loss > val_loss:
        print('val loss decreased, saved model')
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'results/dl_encmclr_dopout0p2_noznorm.pth' )

  with open('results/loss_dl_encmclr_dropout0p2__noznorm.pkl','wb') as f:
    pickle.dump(losses,f)

  return model, model.eval()

In [6]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
model = Enc_MC_LR().to(device) #double?

In [7]:
trained,ev = train_valid(model, device, train_loader, val_loader, class_num=39, input_len=18252, n_epochs=300)

  0%|          | 0/300 [00:00<?, ?it/s]

epoch : 1/300, loss = 3.663340, val_loss = 3.664159
val loss decreased, saved model


  0%|          | 1/300 [00:09<48:53,  9.81s/it]

epoch : 2/300, loss = 3.662934, val_loss = 3.664142
val loss decreased, saved model


  1%|          | 3/300 [00:29<45:32,  9.20s/it]  

...

epoch : 298/300, loss = 3.511082, val_loss = 3.547419


100%|█████████▉| 299/300 [34:07<00:06,  6.30s/it]

epoch : 299/300, loss = 3.510762, val_loss = 3.571260


100%|██████████| 300/300 [34:12<00:00,  6.84s/it]

epoch : 300/300, loss = 3.510644, val_loss = 3.546981


In [68]:
criterion = nn.CrossEntropyLoss()

In [40]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

In [44]:
output

tensor(2.1669, grad_fn=<DivBackward1>)

In [43]:
torch.randn(3, 5, requires_grad=True)

tensor([[ 1.2228, -0.6118, -0.3990, -0.1868, -1.2650],
        [-1.0470, -0.2454, -0.5919,  1.3557,  0.2780],
        [-1.9957, -2.3080,  1.0985,  2.4081, -0.2818]], requires_grad=True)

In [41]:
torch.empty(3, dtype=torch.long).random_(5)

tensor([0, 4, 0])

In [42]:
torch.randn(3, 5).softmax(dim=1)

tensor([[0.3029, 0.0985, 0.3255, 0.1294, 0.1438],
        [0.3081, 0.2550, 0.1167, 0.1676, 0.1527],
        [0.2497, 0.0885, 0.3613, 0.1574, 0.1432]])

In [2]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |